<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.3: 
# *Database Programming with Python and SQLite*

<a name="demo"></a>
## Using SQLite

### Advantages of SQLite

- does not run on a separate server
- creates portable SQL databases saved in a single file
- databases are stored in a very efficient manner and allow fast querying
- ideal for small databases or databases that need to be copied across machines.
- prototyping applications (e.g. as an embedded database server in a Python program).



### The `sqlite3` Command Line Utility

- useful for basic SQL tasks and databse maintenance
- for creating and dropping databases, it may be safer to use the command line than to roll the code into a Python program

1. Add the sqlite3 installation folder to your PATH environment variable.
2. Open a command window.
3. Navigate to your preferred working directory.
4. Start the sqlite3 command line utility and create a database called "ex1":

`$ sqlite3 ex1`

Output:  

`SQLite version 3.8.5 2014-05-29 12:36:14
Enter ".help" for usage hints.
sqlite>`

*Nb. If you leave out the database name (after `sqlite3`, above) a new, temporary database will be created; it will be destroyed upon exiting sqlite3.*  


Enter the following commands at the sqlite prompt (not shown):

`create table tbl1(one varchar(10), two smallint);
insert into tbl1 values('hello!',10);
insert into tbl1 values('goodbye', 20);
select * from tbl1;`

Output:  

`hello!|10
goodbye|20`

Enter `.help` at the sqlite3 command prompt. This lists the available "dot commands". 

Some of the more useful dot commands include:

`  
.backup ?DB? FILE      Backup DB (default "main") to FILE.
.cd DIRECTORY          Change the working directory to DIRECTORY
.clone NEWDB           Clone data into NEWDB from the existing database
.databases             List names and files of attached databases
.dump ?TABLE? ...      Dump the database in an SQL text format.
.excel                 Display the output of next command in a spreadsheet
.exit                  Exit this program
.headers on|off        Turn display of headers on or off
.help                  Show this message
.import FILE TABLE     Import data from FILE into TABLE
.indexes ?TABLE?       Show names of all indexes.
.quit                  Exit this program
.read FILENAME         Execute SQL in FILENAME
.restore ?DB? FILE     Restore content of DB (default "main") from FILE
.save FILE             Write in-memory database into FILE
.tables ?TABLE?        List names of tables.`


### The `sqlite3` package

The easiest way to incorporate an SQL database into a Python application is by using the `sqlite3` package for [`Python 2.7`](https://docs.python.org/2.7/library/sqlite3.html) or [`Python 3.x`](https://docs.python.org/3/library/sqlite3.html).

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [1]:
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db) 

c = conn.cursor()
# c is a way for us to run commands on this database

Commands can be executed by passing them as string arguments to the `execute` method of the cursor we just created for this database.

*Nb. If this script has never been run before, the following will raise an (inconsequential) error message:*

In [2]:
c.execute('DROP TABLE houses;')

# Save (commit) the changes:
conn.commit()

In [3]:
# define table SCHEMA eg field 1 (column) is an integer and primary key, sqft (column) is also integer etc.
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')

conn.commit()

With the database saved the table should now be viewable using SQLite Manager.

#### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

> c.execute(sql_command, values)

In [4]:
# A
last_sale = (None, 4000, 5, 22, 619000)

# Execute Command

# Remember to commit the changes
conn.commit()

Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key. 

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

> c.executemany(sql_command, values)

In [5]:
# A
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

# Nb. The "?" parameters are placeholders for data that will map to the table columns during insertion;
# this is a security measure against SQL injection attacks:
# 

conn.commit()

Select all rows from houses

In [7]:
# A
c.execute("SELECT * FROM houses")




#### Adding data from a csv file
One way to populate the database from a file is to use `numpy.genfromtxt` to read the file into an array (converted to a list for easier handling), and then `INSERT` those records into the database.  The `genfromtxt` function has options including the output data type, handling of missing values, skipping of header and footer rows, columns to read, and more. 

In [8]:
# genfromtxt reads file into numpy array (of integers)
from numpy import genfromtxt

# import into nparray of ints
# convert nparray to list of lists:
data = genfromtxt('housing-data.csv', dtype = 'i8', 
                    delimiter = ',', skip_header = 1).tolist()

In [9]:
pwd

'C:\\Users\\Megan Yit\\Downloads\\SQL Labs'

Suppose we need to put a placeholder in the first column for data that will be available later. Best practice is to insert the value `None`:

In [10]:
# PREPEND a None value to beginning of each sub-list
# (data will be avail later, just create avail space for data first)
for d in data:
    d.insert(0, None)

*Nb. This is why we converted the input array to a list. An array can only hold one type of data (integers in this case) so we could not have inserted `None` before we did this conversion.*

In [11]:
# check:
print(type(data))
print(data[0:3])

<class 'list'>
[[None, 2104, 3, 70, 399900], [None, 1600, 3, 28, 329900], [None, 2400, 3, 44, 369000]]


Now we can insert each list item as a row of fields in the database.

In [12]:
# loop through data, running an INSERT on each record (i.e. sublist):
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

In this case, because we were inserting the same value for all records, so we could have simply used a 'None' in the numpy `insert` method at column 0.

In [17]:
import numpy as np

d1 = np.asarray([1200, 3, 15, 250000])
# np.asarray is used to convert input into array

d1 = d1.tolist()
# tolist() convert series to list

d1.insert(0, None)
d1
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d1)
conn.commit()

#### Deleting Rows

The `DELETE FROM` statement can be used with a `WHERE` clause to specify rows to delete based on some criteria.

In [19]:
c.execute('DELETE FROM houses WHERE field1 IN (52, 53)')
conn.commit()

QUIZ: What would `DELETE FROM houses` do?

In [ ]:
# DELETE FROM houses removes entire rows (that meet criteria) from the houses table

#### Filtering Rows

**1. Select Rows Where Bedrooms = 4**

In [21]:
# ANSWER
results1 = c.execute("SELECT * FROM houses WHERE bdrms = 4")
results1.fetchall()
# fetchall fetches all rows of a query result set and returns a LIST of TUPLES

[(5, 3000, 4, 75, 539900),
 (6, 1985, 4, 61, 299900),
 (11, 1940, 4, 7, 239999),
 (16, 2300, 4, 77, 449900),
 (19, 2609, 4, 5, 499998),
 (20, 3031, 4, 21, 599000),
 (24, 1962, 4, 53, 259900),
 (33, 2040, 4, 75, 314900),
 (35, 1811, 4, 24, 285900),
 (38, 2132, 4, 28, 345000),
 (39, 4215, 4, 66, 549000),
 (40, 2162, 4, 43, 287000),
 (43, 2567, 4, 57, 314000),
 (46, 1852, 4, 64, 299900)]

**2. Run a query to calculate the average floor area and price of each size of house (i.e. by number of bedrooms):**

In [22]:
# ANSWER
results = c.execute("SELECT bdrms, AVG(sqft) AS avg_sqft, AVG(price) AS avg_price FROM houses GROUP BY bdrms")
results.fetchall()

[(1, 1000.0, 169900.0),
 (2, 1496.5, 280866.6666666667),
 (3, 1750.655172413793, 315865.4482758621),
 (4, 2400.4285714285716, 377449.78571428574),
 (5, 4478.0, 699900.0)]

### Pandas connector

While databases provide many analytical capabilities, at some point we may need to pull data into Python for more flexible processing. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales for a report or dashboard, this would be a fixed operation on a large dataset. These computations would run more efficiently in the database system itself.

However, if we wanted to model the patterns of login behaviour or factors driving sales, then we would import the data to Python where we could use its simple interfaces to powerful analytic libraries.

In [25]:
import pandas as pd
from pandas.io import sql

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

### Writing data into a database

Data in Pandas can be loaded into a relational database. 

If the data table is not too large, we can load all of it into a Pandas DataFrame:

In [26]:
# Nb. Use low_memory=False to ensure that type inference does not fail due to buffered processing of input:
data = pd.read_csv('housing-data.csv', low_memory = False)
data.head()

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


We can move data in the opposite direction -- from a DataFrame to a database -- using the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

This copies our `data` DataFrame to a sqlite3 table called `houses_pandas`:

In [27]:
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)

Run a query to get the average price of each house size from this table:

In [31]:
#ANSWER

results2 = c.execute("SELECT bdrms, AVG(price) AS avg_price FROM houses_pandas GROUP BY bdrms")
results2.fetchall()

[(1, 169900.0),
 (2, 280866.6666666667),
 (3, 326403.92),
 (4, 377449.78571428574),
 (5, 699900.0)]

## Discussion

**Scenarios for using Pandas with SQLite**

1. When would you want to use Pandas on a dataset before storing it in a database?

2. When would you want to use Pandas on a dataset retrieved from a database?

In [ ]:
# 1) when we need to clean the data? eg drop unncessary columns in df, changing indexes of df,
# using .str() methods to clean columns, using df.applymap() function to clean entire dataset element-wise,
# renaming columns to more recognizable labels
# skip unncessary rows in CSV files
# (realpython.com)

# 2) when we need to analyse data (EDA essentially, to understand your data at a glimpse)
# or when we need to manipulate the data with flexibility


<a name="guided-practice"></a>


# Reference: SQL Syntax 


#### SELECT Statement
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.

```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE Clause
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is effectively a row filter; rows that match the condition will be included in the rowset that is returned by the query.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for houses older than 60 years.
>```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

### AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate funtions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

### Read Order Data
- P12-ListOfOrders
- P12-OrderBreakdown

#### 1. Read CSV into DataFrame

In [33]:
import pandas as pd
import sqlite3
from pandas.io import sql

In [34]:
# Reading CSV to Dataframe
orders = pd.read_csv('P12-ListOfOrders.csv', encoding = 'utf-8')
orders_break_down =  pd.read_csv('P12-OrderBreakdown.csv', encoding = 'utf-8')

In [35]:
orders.head()

,Order ID,Order Date,Customer Name,Country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [36]:
orders_break_down.head()

,Order ID,Product Name,Discount,Sales,Quantity,Category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


#### 2. Replace Space with Underscore in Column Names

In [37]:
# ANSWER
orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]

In [39]:
# ANSWER
orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]

#### 3. Check DataTypes 

In [40]:
# ANSWER
orders.dtypes

order_id         object
order_date       object
customer_name    object
country          object
dtype: object

In [41]:
# ANSWER
orders_break_down.dtypes

order_id         object
product_name     object
discount        float64
sales           float64
quantity          int64
category         object
dtype: object

#### 4. Save these two dataframes as a table in sqlite

In [43]:
# Establishing Local DB connection
db_connection = sqlite3.connect('eshop.db.sqlite')

# Save these two dataframes as a table in sqlite
orders.to_sql('orders_pandas', con = db_connection, if_exists = 'replace', index = False)
orders_break_down.to_sql('orders_break_down_pandas', con = db_connection, if_exists = 'replace', index = False)

#### 5. Select Number of Orders for Each Customer

We can use the `sql` function in `pandas.io` to run SQL queries to the database with the `.read_sql()` method. Here's an example:

In [46]:
# Select first 10 rows of all columns in orders table

eg_query = """
        SELECT * 
        FROM orders 
        LIMIT 10;
        """

query = "SELECT * FROM orders_pandas LIMIT 10"

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France
5,ES-2011-5433855,2011-01-07,Dave Poirier,France
6,IT-2011-4546695,2011-01-08,Darren Powers,France
7,ES-2011-1138719,2011-01-11,Eric Murdock,Italy
8,ES-2011-1466305,2011-01-11,Mick Brown,Austria
9,ES-2011-4359424,2011-01-11,Dorothy Dickinson,Spain


In [85]:
# Select Number of Orders for Each Customer
query1 = "SELECT customer_name, COUNT(*) AS order_count_per_customer FROM orders_pandas GROUP BY customer_name ORDER BY order_count_per_customer DESC"

orders_count_per_cust = sql.read_sql(query1, con=db_connection)
orders_count_per_cust

,customer_name,order_count_per_customer
0,John Grady,13
1,Joel Jenkins,12
2,Aaron Smayling,12
3,Yoseph Carroll,11
4,Sarah Brown,11
...,...,...
787,Brooke Gillingham,1
788,Barbara Fisher,1
789,Art Ferguson,1
790,Andy Reiter,1


#### 6. Select Number of Customers for Each Country

In [89]:
# Select Number of Customers for Each Country
query2 = "SELECT country, COUNT(*) AS cust_count_per_country FROM orders_pandas GROUP BY country ORDER BY cust_count_per_country DESC"

customer_count_per_country = sql.read_sql(query2, con=db_connection)
customer_count_per_country

,country,cust_count_per_country
0,France,991
1,Germany,806
2,United Kingdom,700
3,Italy,493
4,Spain,403
5,Netherlands,194
6,Austria,135
7,Sweden,100
8,Belgium,68
9,Ireland,50


#### 7.A Select discount, sales, quantity for Each Order from orders_break_down Table

In [54]:
#  Select discount, sales, quantity for Each Order from orders_break_down Table

query3 = "SELECT order_id, discount, sales, quantity FROM orders_break_down_pandas"

discount_sales_qty = sql.read_sql(query3, con=db_connection)
discount_sales_qty 

,order_id,discount,sales,quantity
0,IT-2011-3647632,0.5,45.0,3
1,ES-2011-4869686,0.0,854.0,7
2,ES-2011-4939443,0.0,140.0,3
3,IT-2011-2942451,0.5,27.0,2
4,IT-2011-2942451,0.5,17.0,2
...,...,...,...,...
8042,ES-2014-3458802,0.0,245.0,2
8043,IT-2014-3715679,0.5,30.0,2
8044,IT-2014-3715679,0.5,23.0,4
8045,IT-2014-3715679,0.5,108.0,3


#### 7.B Select discount, sales, quantity, total price for Each Order from orders_break_down Table

> Total Price = sales * quantity - discount

In [58]:
# Select discount, sales, quantity, total price for Each Order from orders_break_down Table

query4 = "SELECT order_id, discount, sales, quantity, ((sales*quantity) - discount) AS total_price FROM orders_break_down_pandas"

discount_sales_qty_totalprice = sql.read_sql(query4, con=db_connection)
discount_sales_qty_totalprice

,order_id,discount,sales,quantity,total_price
0,IT-2011-3647632,0.5,45.0,3,134.5
1,ES-2011-4869686,0.0,854.0,7,5978.0
2,ES-2011-4939443,0.0,140.0,3,420.0
3,IT-2011-2942451,0.5,27.0,2,53.5
4,IT-2011-2942451,0.5,17.0,2,33.5
...,...,...,...,...,...
8042,ES-2014-3458802,0.0,245.0,2,490.0
8043,IT-2014-3715679,0.5,30.0,2,59.5
8044,IT-2014-3715679,0.5,23.0,4,91.5
8045,IT-2014-3715679,0.5,108.0,3,323.5


#### 7.C Select All Orders from orders_break_down Table Where Total Price Greater Than 100

In [60]:
# Select All Orders from orders_break_down Table Where Total Price Greater Than 100

query5 = "SELECT order_id, ((sales*quantity) - discount) AS total_price FROM orders_break_down_pandas WHERE total_price > 100"

totalprice_morethan100 = sql.read_sql(query5, con=db_connection)
totalprice_morethan100

,order_id,total_price
0,IT-2011-3647632,134.5
1,ES-2011-4869686,5978.0
2,ES-2011-4939443,420.0
3,ES-2011-3848439,270.0
4,ES-2011-3848439,828.0
...,...,...
6461,ES-2014-2815584,1566.0
6462,ES-2014-4785777,160.0
6463,ES-2014-3458802,490.0
6464,IT-2014-3715679,323.5


#### 8. Select All Customer And The Product They Have Bought

In [62]:
# Select All Customer And The Product They Have Bought

query6 = "SELECT orders.order_id, orders.customer_name, orders_bd.product_name FROM orders_pandas orders JOIN orders_break_down_pandas orders_bd ON orders.order_id = orders_bd.order_id"

cust_product = sql.read_sql(query6, con=db_connection)
cust_product

,order_id,customer_name,product_name
0,IT-2011-3647632,Eugene Moren,"Enermax Note Cards, Premium"
1,ES-2011-4869686,Dorothy Dickinson,"Dania Corner Shelving, Traditional"
2,ES-2011-4939443,Arthur Prichep,"Binney & Smith Sketch Pad, Easy-Erase"
3,IT-2011-2942451,Grant Thornton,"Boston Markers, Easy-Erase"
4,IT-2011-2942451,Grant Thornton,"Eldon Folders, Single Width"
...,...,...,...
8042,ES-2014-3638865,Susan Vittorini,"Fellowes Shelving, Single Width"
8043,ES-2014-4785777,Dennis Pardue,"Wilson Jones Index Tab, Economy"
8044,IT-2014-3715679,Jim Kriz,"Avery Binder Covers, Recycled"
8045,IT-2014-3715679,Jim Kriz,"BIC Pencil Sharpener, Fluorescent"


#### 9.A Select Number of 'Furniture' Orders For Each Country 

In [102]:
# Select Number of 'Furniture' Orders For Each Country

query7 ="""
        SELECT COUNT(*) AS furniture_count_per_country,
        orders.country
        FROM orders_pandas orders
        JOIN orders_break_down_pandas orders_bd
        ON orders.order_id = orders_bd.order_id
        AND orders_bd.category = 'Furniture'
        GROUP BY orders.country
        """
        

furniture_count_country = sql.read_sql(query7, con=db_connection)
furniture_count_country

,furniture_count_per_country,country
0,40,Austria
1,20,Belgium
2,9,Denmark
3,16,Finland
4,299,France
5,264,Germany
6,12,Ireland
7,136,Italy
8,76,Netherlands
9,6,Norway


#### 9.B Select Number of 'Furniture' Orders For The Country Denmark

In [104]:
# Select Number of 'Furniture' Orders For The Country Denmark

query8 = """
        SELECT orders.country,
        COUNT(1) AS furniture_count
        FROM orders_pandas orders
        JOIN orders_break_down_pandas orders_bd
        ON orders.order_id = orders_bd.order_id
        AND orders_bd.category == "Furniture"
        AND orders.country == "Denmark"
        """

furniture_count_denmark = sql.read_sql(query8, con=db_connection)
furniture_count_denmark

,country,furniture_count
0,Denmark,9


#### 10. Select Total Sales With Discount and Without Discount for Each Country

In [137]:
# Select Total Sales With Discount and Without Discount for Each Country
# Total Price = sales * quantity - discount

query9 = """
        SELECT orders.country,
        SUM(orders_bd.sales*quantity) AS total_sales_bef_discount,
        SUM(orders_bd.sales*quantity - discount) AS total_sales_after_discount
        FROM orders_pandas orders
        JOIN orders_break_down_pandas orders_bd
        ON orders.order_id = orders_bd.order_id
        GROUP BY orders.country
        """

total_sales_per_country = sql.read_sql(query9, con=db_connection)
total_sales_per_country


,country,total_sales_bef_discount,total_sales_after_discount
0,Austria,391694.0,391694.00
1,Belgium,229829.0,229829.00
2,Denmark,34617.0,34586.50
3,Finland,82001.0,82001.00
4,France,3108592.0,3108445.00
5,Germany,2411483.0,2411393.15
6,Ireland,75724.0,75673.70
7,Italy,1177921.0,1177798.40
8,Netherlands,379714.0,379525.10
9,Norway,99878.0,99878.00


#### 11.A Select Total Quantity, Total Sales for Each Country

In [135]:
# Select Total Quantity, Total Sales for Each Country

query10 = """
        SELECT orders.country,
        SUM(orders_bd.quantity) AS total_qty,
        SUM(orders_bd.sales) AS total_sales
        FROM orders_pandas orders
        JOIN orders_break_down_pandas orders_bd
        ON orders.order_id = orders_bd.order_id
        GROUP BY orders.country
        ORDER BY 3 DESC
        """

total_qtysales_per_country = sql.read_sql(query10, con=db_connection)
total_qtysales_per_country

,country,total_qty,total_sales
0,France,7329,609683.0
1,Germany,6179,488681.0
2,United Kingdom,4917,420497.0
3,Italy,3612,252742.0
4,Spain,2881,249402.0
5,Austria,973,79382.0
6,Netherlands,1526,70313.0
7,Belgium,532,42320.0
8,Sweden,753,30490.0
9,Switzerland,308,24874.0


#### 11.B Select Top 3 Country Based on Sales

In [131]:
# Select Top 3 Country Based on Sales

query11 = """
        SELECT orders.country,
        SUM(orders_bd.sales) AS total_sales
        FROM orders_pandas orders
        JOIN orders_break_down_pandas orders_bd
        ON orders.order_id = orders_bd.order_id        
        GROUP BY orders.country
        ORDER BY 2 DESC
        LIMIT 3
        """

top_3_sales_country = sql.read_sql(query11, con=db_connection)
top_3_sales_country

,country,total_sales
0,France,609683.0
1,Germany,488681.0
2,United Kingdom,420497.0


#### 11.C Select Bottom 3 Country Based On Quantities

In [129]:
# Select Bottom 3 Country Based On Quantities

query12 = """
        SELECT orders.country,
        SUM(orders_bd.quantity) AS total_qty
        FROM orders_pandas orders
        JOIN orders_break_down_pandas orders_bd
        ON orders.order_id = orders_bd.order_id        
        GROUP BY orders.country
        ORDER BY 2
        LIMIT 3
        """

bottom_3_qty_country = sql.read_sql(query12, con=db_connection)
bottom_3_qty_country

,country,total_qty
0,Finland,201
1,Denmark,204
2,Norway,261


#### 12. Select Average Sales By Categroy For The Country 'France'

In [126]:
# Select Average Sales By Categroy For The Country 'France'

query13 = """
        SELECT orders.country,
        orders_bd.category,
        SUM(orders_bd.sales)
        FROM orders_pandas orders
        JOIN orders_break_down_pandas orders_bd
        ON orders.order_id = orders_bd.order_id        
        AND orders.country == 'France'
        GROUP BY orders.country, orders_bd.category
        ORDER BY SUM(orders_bd.sales)
        """

avg_sales_france_by_category = sql.read_sql(query13, con=db_connection)
avg_sales_france_by_category

,country,category,SUM(orders_bd.sales)
0,France,Furniture,174112.0
1,France,Office Supplies,205845.0
2,France,Technology,229726.0


#### 13. Select Country, Category and Total Sales Where Average Total Sales is The Highest

In [143]:
# Select Country, Category and Total Sales Where Average Total Sales is The Highest

query14 = """
        SELECT orders.country,
        orders_bd.category,
        AVG(orders_bd.sales) AS avg_sales
        FROM orders_pandas orders
        JOIN orders_break_down_pandas orders_bd
        ON orders.order_id = orders_bd.order_id
        GROUP BY orders.country, orders_bd.category
        ORDER BY 3 DESC
        LIMIT 1
        """

highest_avg_sales = sql.read_sql(query14, con=db_connection)
highest_avg_sales

,country,category,avg_sales
0,Switzerland,Technology,902.846154


### JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams. 
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

### ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [sqlite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
- [SQLite Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  
- [SQL zoo](http://www.sqlzoo.net)  Great for learning syntax



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



